In [ ]:

!pip install easyocr opencv-python -q


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import easyocr
import warnings
warnings.filterwarnings('ignore')

# 스타일
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 5)

# 경로
BASE_DIR = '/home/realtheai/cv_competetion'
TRAIN_CSV = f'{BASE_DIR}/data/train.csv'
TEST_CSV = f'{BASE_DIR}/data/sample_submission.csv'
TRAIN_IMG_DIR = f'{BASE_DIR}/data/train'
TEST_IMG_DIR = f'{BASE_DIR}/data/test'
META_CSV = f'{BASE_DIR}/data/meta.csv'

# 데이터 로드
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
meta_df = pd.read_csv(META_CSV)
class_names = dict(zip(meta_df['target'], meta_df['class_name']))

print(f"✅ Train: {len(train_df)} | Test: {len(test_df)} | Classes: {len(meta_df)}")


## 1. Train vs Test 품질 비교 


In [ ]:
def analyze_quality(img_path):
    """이미지 품질 지표 계산"""
    img = cv2.imread(img_path)
    if img is None:
        return None
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return {
        'brightness': np.mean(gray),
        'contrast': np.std(gray),
        'sharpness': cv2.Laplacian(gray, cv2.CV_64F).var()
    }

def collect_stats(img_dir, df, n=100):
    """통계 수집"""
    stats = []
    for img_name in tqdm(df['ID'].sample(min(n, len(df))), desc=img_dir.split('/')[-1]):
        img_path = os.path.join(img_dir, img_name)
        stat = analyze_quality(img_path)
        if stat:
            stats.append(stat)
    return pd.DataFrame(stats)

# 수집
train_stats = collect_stats(TRAIN_IMG_DIR, train_df)
test_stats = collect_stats(TEST_IMG_DIR, test_df)

# 비교
print("\n📊 Train vs Test 품질 비교")
print("="*60)
for metric in ['brightness', 'contrast', 'sharpness']:
    t_mean = train_stats[metric].mean()
    s_mean = test_stats[metric].mean()
    diff = abs(t_mean - s_mean)
    print(f"{metric:12s} | Train: {t_mean:6.1f} | Test: {s_mean:6.1f} | Diff: {diff:6.1f}")
print("="*60)


In [ ]:
# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['brightness', 'contrast', 'sharpness']):
    axes[idx].hist(train_stats[metric], bins=20, alpha=0.6, label='Train', color='blue')
    axes[idx].hist(test_stats[metric], bins=20, alpha=0.6, label='Test', color='red')
    axes[idx].set_title(metric.capitalize())
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{BASE_DIR}/05_train_vs_test_quality.png', dpi=150, bbox_inches='tight')
plt.show()


## 2. OCR 키워드 분석 (Class 3, 7, 14)


In [ ]:
# OCR 초기화
print("📖 OCR Reader 로딩...")
reader = easyocr.Reader(['ko', 'en'], gpu=False)
print("✅ 완료!")

def extract_title(img_path, reader):
    """상단 25% 영역에서 텍스트 추출"""
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    
    height = img_array.shape[0]
    title_region = img_array[:int(height * 0.25), :]
    
    try:
        results = reader.readtext(title_region)
        return ' '.join([r[1] for r in results])
    except:
        return ""


In [ ]:
# 의료 문서 분석
medical_classes = [3, 7, 14]
keywords = {
    3: ['입퇴원', '확인서', '입원', '퇴원'],
    7: ['진료', '확인서', '외래'],
    14: ['소견서', '의견', '소견']
}

print("\n🔍 의료 문서 OCR 분석")
print("="*80)

ocr_results = {}

for class_id in medical_classes:
    samples = train_df[train_df['target'] == class_id].head(5)
    
    print(f"\nClass {class_id}: {class_names[class_id]}")
    print("-" * 60)
    
    texts = []
    for _, row in samples.iterrows():
        img_path = os.path.join(TRAIN_IMG_DIR, row['ID'])
        text = extract_title(img_path, reader)
        texts.append(text)
        print(f"  {text[:50]}...")
    
    ocr_results[class_id] = texts
    
    # 키워드 매칭
    all_text = ' '.join(texts)
    found = [kw for kw in keywords[class_id] if kw in all_text]
    
    if found:
        print(f"  ✅ 키워드 발견: {found}")
    else:
        print(f"  ❌ OCR 실패! 키워드 없음")

print("\n" + "="*80)


## 3. 문서 구조 분석 (표/도장)


In [ ]:
def analyze_structure(img_path):
    """상단/중간/하단 엣지 밀도 계산"""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    
    h = img.shape[0]
    top = img[:int(h*0.2), :]
    mid = img[int(h*0.2):int(h*0.8), :]
    bot = img[int(h*0.8):, :]
    
    return {
        'top': cv2.Canny(top, 50, 150).mean(),
        'mid': cv2.Canny(mid, 50, 150).mean(),
        'bot': cv2.Canny(bot, 50, 150).mean()
    }

# 분석
print("\n📐 문서 구조 분석")
print("="*60)

structure_stats = {}

for class_id in medical_classes:
    samples = train_df[train_df['target'] == class_id].head(10)
    
    densities = []
    for _, row in samples.iterrows():
        img_path = os.path.join(TRAIN_IMG_DIR, row['ID'])
        struct = analyze_structure(img_path)
        if struct:
            densities.append(struct)
    
    avg = pd.DataFrame(densities).mean()
    structure_stats[class_id] = avg
    
    print(f"Class {class_id}: Top {avg['top']:.1f} | Mid {avg['mid']:.1f} | Bot {avg['bot']:.1f}")

print("="*60)


In [ ]:
# 시각화
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(3)
width = 0.25

for i, class_id in enumerate(medical_classes):
    values = [structure_stats[class_id][pos] for pos in ['top', 'mid', 'bot']]
    ax.bar(x + i*width, values, width, label=f'Class {class_id}')

ax.set_xlabel('Document Region')
ax.set_ylabel('Edge Density')
ax.set_title('Document Structure: Class 3 vs 7 vs 14')
ax.set_xticks(x + width)
ax.set_xticklabels(['Top (Title)', 'Middle (Body)', 'Bottom (Stamp)'])
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{BASE_DIR}/05_document_structure.png', dpi=150, bbox_inches='tight')
plt.show()
